In [3]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
from fake_useragent import UserAgent
from urllib.parse import urlencode
from tabulate import tabulate
import mysql.connector
from mysql.connector import Error
from datetime import datetime
import sys

def my_print(ll):
  print(tabulate(ll, tablefmt="grid", stralign="left"))
# ------------------------------конец процедуры для красивого вывода -----------------------------------

ua = UserAgent() # Генерируем случайный User-Agent
headers = {
  'User-Agent': ua.random,
  'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
  'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
  'Connection': 'keep-alive',
  'Upgrade-Insecure-Requests': '1',
  'Sec-Fetch-Dest': 'document',
  'Sec-Fetch-Mode': 'navigate',
  'Sec-Fetch-Site': 'same-origin',
  'Sec-Fetch-User': '?1',
  'Cache-Control': 'max-age=0'
}
session = requests.Session() # Создаем сессию для сохранения кук

def add_to_menu(ur,ff):
  print('передали такие параметры в процедуру:')
  print(f'ur={ur}')
  print(f'ff={ff}')

  time.sleep(6) # Добавляем человеческую задержку
  resp = session.get(
    ur,
    headers=headers,
    timeout=10
  )
  #time.sleep(13) # Добавляем человеческую задержку
  if resp.status_code == 200: # Проверяем ответ
    print(f"получен ответ200! Сервер ответил успешно.")
    # Дальнейшая обработка HTML...
  else:
    print(f"Ошибка: {resp.status_code}")
    sys.exit(4) #если нет нужного ответа, то останавливаемся

  soup= BeautifulSoup(resp.text, 'html.parser')
  sel= soup.find('select', {'name': 'group'})
  if sel:
    #print(f"Найден элемент select с именем 'group': {sel}")
    opts     = sel.find_all('option')
    #print(f'dlina opts={len(opts)}')
    act_opts = [opt for opt in opts if not opt.get('disabled')]
  else:
    print(f'не смог найти элемент SELECT, это там, где все конкурсы хранятся.:')
    print(sel)
    print('весь суп:')
    print(soup)
    print("-" * 50)
    print(resp.text)
    sys.exit(4) #че-то не то, останавливаемся
  if len(opts) != len(act_opts):
    print (f"количество значений в options не равняется options, для которых отсутствует disabled. по-идее, так быть не должно opt={len(opts)} act_options{len(act_opts)}")
    sys.exit(4) #че-то не то, останавливаемся
  else:
    print (f"все норм. форма={ff} opt={len(opts)} act_options{len(act_opts)}")

  print("-" * 50)
  #print(opts)
  for i,o1o in enumerate(act_opts,1):
    val=o1o.get('value','hz')
    #print(f"{i} -- {val}")
    if val != '':
      menu_opt.append({"forma":ff,"name_konk":val})
# -------------------------- конец процедуры формирования меню ----------------------------------------------------------------------

def get_konk(name_konk, priz): #должна вернуть суп для разбора
  delay = random.uniform(10, 20)
  print(f"Пауза {delay:.1f} сек перед обработкой {name_konk}")
  time.sleep(delay)
  params = {
    "valueSearch": "",
    "priznakViev":priz,
    "levelTarget": "bak_main",
    "group": name_konk,
    "originalFilter": "",
    "paid": "",
    "search_type": "uniqueID",
    "originalView": "all"
  }
  resp2 = session.get(
    url2,
    params=params,
    headers=headers,
    timeout=30
  )
  if resp2.status_code == 200: # Проверяем ответ
    print(f"Успешный запрос! Сервер ответил успешно и должен выдать список {name_konk}.")
    encoded_params = urlencode(params, encoding='utf-8')
    full_url = f"{url2}?{encoded_params}"
    print(f"URL для браузера: {full_url}")# Выводим URL, готовый для использования в браузере
  else:
    print(f"Ошибка: {resp2.status_code}")
    sys.exit(5) #если нет нужного списка, то останавливаемся
  return BeautifulSoup(resp2.text, 'html.parser')
# -------------------------------------- конец процедуры get_konk ---------------------------------------------------

def konk_info():
  konkurs_tag = soup.find('ul', class_='list-head') # Находим элемент ul с классом "list-head" - походу тут хранится инфо о конкурсе
  #print(konkurs_tag)
  if not konkurs_tag: # Если элемент не найден
    print(f"Элемент ul.list-head не найден на странице, здесь должна быть информация о конкретном конкурсе {values[0]}")
    sys.exit(5) #если нет нужного контейнера, то останавливаемся
  konkurs_li = konkurs_tag.find_all('li') # получили резалтсет с информацией о конкурсе, пока что тут еще обернуто в li и в span
  konkurs_opis = [] #тут будет чистая информация описания конкурса для записи в БД безо всяких лишних тэгов
  for konkurs_info in konkurs_li:
    konkurs_opis.append(konkurs_info.find('span').get_text(strip=True))  # сформирован list с информацией о конкурсе
  #print(konkurs_opis)
  return konkurs_opis
# --------------------------- конец процедуры konk_info ---------- возвращает список с информацией о конкурсе ------------------------------

def konk_data():
  konkurs_data = [] #тут будет чистая информация по людям конкурса для записи в БД безо всяких лишних тэгов

  def mtusi_format1(zn): # формат всегда к числу
    if zn == "":
      return 0
    try:
      return int(zn)
    except ValueError:
      return -1

  tb = soup.find('tbody').find_all('tr')
  #print(tb[9])
  #print(tb[49])
  #print(tb[50])
  #print(tb[51])
  for tr_row1 in tb:
    if 'Отсутствуют свободные места' in tr_row1.find('td').get_text():
      continue #адская верстка
    td_row1 = tr_row1.find_all('td')
    td_data_cell_row1 = [td_data_cell1.get_text(strip=True) for td_data_cell1 in td_row1 ]
    td_data_cell_row1.append({'bvi':0})
    td_data_cell_row1.append({'yz':0})
    if tr_row1.get('data-bvi') != '0': #обрабатываем БВИ
      #print ('чото есть бви')
      td_data_cell_row1[-2]['bvi'] = int(tr_row1.get('data-bvi'))
      td_data_cell_row1[3] = '333' #здесь итог баллов
      td_data_cell_row1[4] = '100' #баллы за 1й экзамен, т.к. в хтмл разметке есть слияние ячеек colspan3
      td_data_cell_row1.insert(5,'100') #баллы за 2й экзамен, т.к. в хтмл разметке есть слияние ячеек colspan3
      td_data_cell_row1.insert(6,'100') #баллы за 3й экзамен, т.к. в хтмл разметке есть слияние ячеек colspan3
    if tr_row1.get('style') == "background-color: #7765ba; color: white;": #обрабатываем фиолетовый цвет, якобы зачислен
      td_data_cell_row1[-1]['yz'] = 1

    for ii in [0,1,2,3,4,5,6,7,9,10,11]:
      td_data_cell_row1[ii] = mtusi_format1(td_data_cell_row1[ii])
    konkurs_data.append(td_data_cell_row1)
  return konkurs_data
# ---------------------------------- конец процедуры konk_data ------------ здесь формируются данные по людям -----------------------------------------

def conn(): #коннект  к бд
  print(f"{datetime.now()} метод коннекта к бд начало.")
  global connection

  if connection is not None and connection.is_connected():
    print('пытались выполнить метод connect, но коннект то уже есть. выходим')
    return connection
  try:
    connection = mysql.connector.connect(host='localhost',
      database='vuz',
      user='forpiton',
      password='passwordpiton',
      autocommit=True  # Рекомендуется для большинства случаев
    )
    if connection.is_connected():
      print(f"  {datetime.now()}| Успешно подключено к базе данных")
      return connection
  except Error as e:
    print(f"  {datetime.now()}| Ошибка в методе CONN при подключении к MySQL: {e}")
    sys.exit(4)
  return None

def disc(): #дисконнект воркера к бд
  global connection
  print(f"{datetime.now()}|  метод дисконнекта от бд начало, conn= {connection}")
  try:
    if connection is not None and connection.is_connected():
      connection.close()
      print(f"  {datetime.now()}|  Соединение с MySQL было закрыто")
      connection = None  # Сбрасываем глобальную переменную
    else:
      print(f'  {datetime.now()}| в методе дисконнект на найдено открытого подключения к субд')
  except Error as e:
    print(f"{datetime.now()}| Ошибка в методе DISCONNECT при отключении от MySQL: {e}")
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ конец двух процедур коннекта и дисконнекта к БД ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

def mtusi_insert_data(id_sess,nomerpp, bvi, gu_kod, n_dela, sum_bal, bal_inf,bal_matem,bal_rus,bal_id,soglasie,prior,v_prior,p_prior,obsh,prem_pravo,prem_pravo2,zach,osn_bvi,konk_name, konk_kateg, konk_napravl, konk_profil,konk_svobodno,konk_forma,konk_osnov,konk_uroven,konk_mest,yz):
  #Параметризованный INSERT запрос
  try:
    insert_query = """
            INSERT INTO ludi (id_sess,nomerpp, bvi, gu_kod, n_dela, sum_bal,bal_inf,bal_matem,bal_rus,bal_id,soglasie,prior,v_prior,p_prior,obsh,prem_pravo,prem_pravo2,zach,osn_bvi,konk_name, konk_kateg, konk_napravl, konk_profil,konk_svobodno,konk_forma,konk_osnov,konk_uroven,konk_mest,yz)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
    global connect
    cursor = connect.cursor()
    cursor.execute(insert_query, (id_sess,nomerpp, bvi, gu_kod, n_dela, sum_bal,bal_inf,bal_matem,bal_rus,bal_id,soglasie,prior,v_prior,p_prior,obsh,prem_pravo,prem_pravo2,zach,osn_bvi,konk_name, konk_kateg,konk_napravl, konk_profil,konk_svobodno,konk_forma,konk_osnov,konk_uroven,konk_mest,yz)) #ниче не коммитим, т.к. в парметрах подключения стоит автокоммит
    cursor.close()
    return True
  except Error as e:
    print(f"  Ошибка INSERT: {e}")
    print(id_sess,nomerpp, bvi, gu_kod, n_dela, sum_bal, bal_inf,bal_matem,bal_rus,bal_id,soglasie,prior,v_prior,p_prior,obsh,prem_pravo,prem_pravo2,zach,osn_bvi,konk_name, konk_kateg, konk_napravl, konk_profil,konk_svobodno,konk_forma,konk_osnov,konk_uroven,konk_mest,yz)
    sys.exit(5)
  finally:
    pass # Не закрываем соединение здесь, чтобы можно было выполнить другие запросы
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ конец процедуры, которая пишет данные в бд ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

# ------- сформируем уникальный идентификатор нашей сессии. это будет количество минут от 1го августа 2025 года. всегда увеличивается и значение небольшое -----------------------------
target_date = datetime(2025, 8, 1)
current_date = datetime.now() # Получаем текущую дату и время
time_difference = current_date - target_date # Вычисляем разницу между текущей датой и целевой датой
minutes_session = int(time_difference.total_seconds() / 60) # Преобразуем разницу в минуты и выводим
print(f"Количество прошедших минут с 1 августа 2025 года: {minutes_session}")
# ------- конец формирования уникального идентификатора нашей сессии.  -----------------------------

# url = "https://list.mtuci.ru/spisok.php?levelTarget=bak_main" #был вначале такой урл, потом сменили
# url = "https://abitur.mtuci.ru/ranked_lists/competition_lists/?valueSearch=&levelTarget=bak_main"
url_b = "https://abitur.mtuci.ru/ranked_lists/competition_lists/?valueSearch=&levelTarget=bak_main&priznakViev=budg" #бюджет 02-08-2025
url_p = "https://abitur.mtuci.ru/ranked_lists/competition_lists/?valueSearch=&levelTarget=bak_main&priznakViev=pvz" #платники 02-08-2025
url2 = "https://abitur.mtuci.ru/ranked_lists/competition_lists/" #это базовый урл для запроса каждого конкурса в отдельности
connection = None #здесь будем хранить коннект к БД

menu_opt = [] #пустой список, тут будут значения меню(конкурсов, которые удалось добыть) и для платников, и для бюджета
ki = None # глобальная переменная для хранения данных по конкурсу (текущий обрабатываемый конкурс. перезаписывается каждый раз для нового конкурса)
kd = None #глобальная перменная для хранения данных по людям в конкурсе. (текущий обрабатываемый конкурс. перезаписывается каждый раз для нового конкурса)
konkursy = [] #здесь будут храниться данные по конкурсам в виде "инфо о конкурсе"-"данные конкурса". такая пара значений для каждого конкурса

add_to_menu(url_b,"budg")
add_to_menu(url_p,"pvz")

my_print(menu_opt)
print(len(menu_opt))
if len(menu_opt)<5:
  print ('что-то пунктов меню меньше, чем было. нужен ручной контроль')
  sys.exit(4) #че-то не то, останавливаемся
print('пока что все ок, продолжаем дальше')

for i,mm in enumerate(menu_opt):
  #if i>1: #для отладки 2 конкурса
    #break
  soup = get_konk(mm['name_konk'],mm['forma']) #это потом нужно забросить в цикл по меню
  print (mm['name_konk'],mm['forma'])
  print(i)
  ki = konk_info()# глобальная переменная для хранения данных по конкурсу
  kd = konk_data()
  konkursy.append((ki,kd))

connect = conn() #подключаемся к БД ща будем мощно инсертить
for konk in konkursy:
  if (konk[0][3] =='Целевой приём') or (konk[0][2] =='Имеющие особое право'):
    continue #не будем загружать квотников
  obrabotano = 0
  for row in konk[1]:
    #print(row)
    b_nomer = row[0]
    nomer = b_nomer
    b_bvi = 1 if row[-2]['bvi'] == 1 else 0
    bvi = b_bvi
    b_gu_kod = row[1]
    gu_kod = b_gu_kod
    b_n_dela = row[2]
    n_dela = b_n_dela
    b_sum_bal = row[3]
    sum_bal = b_sum_bal
    b_bal_inf = row[4]
    bal_inf = b_bal_inf
    b_bal_matem = row[5]
    bal_matem = b_bal_matem
    b_bal_rus = row[6]
    bal_rus = b_bal_rus
    b_bal_id = row[7]
    bal_id = b_bal_id
    b_soglasie = 1 if row[8] =='Да' else 0
    soglasie = b_soglasie
    b_prior = row[9]
    prior = b_prior
    b_v_prior = row[10]
    v_prior = b_v_prior
    b_p_prior = row[11]
    p_prior = b_p_prior
    sb_obsh = row[12]
    b_obsh = 1 if sb_obsh == '√' else 0
    obsh = b_obsh
    b_prem_pravo = 1 if row[13] == '√' else 0
    prem_pravo = b_prem_pravo
    b_prem_pravo2 = 1 if row[14] == '√' else 0
    prem_pravo2 = b_prem_pravo2
    b_zach =0 if row[-4] == "" else 1
    zach =b_zach
    b_osn_bvi = row[-3]
    osn_bvi = b_osn_bvi
    konk_name=konk[0][0]
    konk_kateg=konk[0][2]
    konk_napravl=konk[0][4]
    konk_profil=konk[0][6]
    konk_svobodno = konk[0][-1]
    konk_forma = konk[0][1]
    konk_osnov = konk[0][3]
    if konk_osnov=='Полное возмещение затрат' and not ('ИНО' in konk_name): #другой формат немного таблицы для платников
      prior = b_v_prior
      v_prior = b_p_prior
      p_prior = sb_obsh
      #print(f'nomer={nomer},gu_kod={gu_kod},n_dela={n_dela},sum_bal={sum_bal},bal_inf={bal_inf},bal_matem={bal_matem},bal_rus={bal_rus},bal_id={bal_id},soglasie={soglasie},prior={prior},v_prior={v_prior},p_prior={p_prior},zach={zach}')
      p_prior = 0 if sb_obsh =='' else int(sb_obsh)
    if 'ИНО' in konk_name:
      bal_matem=b_bal_inf
      bal_rus=b_bal_matem
      bal_id = b_bal_rus
      soglasie = -2
      prior = b_prior
      v_prior = b_v_prior
      p_prior = b_p_prior

    konk_uroven = konk[0][-4]
    konk_mest = konk[0][-2]
    yz = 1 if row[-1]['yz'] == 1 else 0
    #print(f'nomer={nomer},gu_kod={gu_kod},n_dela={n_dela},sum_bal={sum_bal},bal_inf={bal_inf},bal_matem={bal_matem},bal_rus={bal_rus},bal_id={bal_id},soglasie={soglasie},prior={prior},v_prior={v_prior},p_prior={p_prior},zach={zach}')
    mtusi_insert_data(minutes_session,nomer,bvi,gu_kod,n_dela,sum_bal,bal_inf,bal_matem,bal_rus,bal_id,soglasie,prior,v_prior,p_prior,obsh,prem_pravo,prem_pravo2,zach,osn_bvi, konk_name,konk_kateg,konk_napravl, konk_profil,konk_svobodno,konk_forma,konk_osnov,konk_uroven,konk_mest,yz)
    obrabotano = obrabotano+1
  print(f'закончил вставку для конкурса {konk[0][0]}, вставил {obrabotano}')
disc() # отключаемся от БД
print('программа ЗАВЕРШИЛАСЬ')

Количество прошедших минут с 1 августа 2025 года: 17776
передали такие параметры в процедуру:
ur=https://abitur.mtuci.ru/ranked_lists/competition_lists/?valueSearch=&levelTarget=bak_main&priznakViev=budg
ff=budg
получен ответ200! Сервер ответил успешно.
все норм. форма=budg opt=87 act_options87
--------------------------------------------------
передали такие параметры в процедуру:
ur=https://abitur.mtuci.ru/ranked_lists/competition_lists/?valueSearch=&levelTarget=bak_main&priznakViev=pvz
ff=pvz
получен ответ200! Сервер ответил успешно.
все норм. форма=pvz opt=41 act_options41
--------------------------------------------------
+------+--------------------------------------------------------------------------------------------------------------------------------------------------+
| budg | 09.03.01 Инженерия систем искусственного интеллекта (Бюджет, Очная) (2025)                                                                       |
+------+---------------------------------------------

In [ ]:
disc()

In [ ]:
soup = get_konk('01.03.02 Прикладная математика и программирование (Платное, Очная) (2025)', 'pvz')
kd = konk_data()

In [ ]:
print(kd[3])

In [ ]:
for i,k in enumerate(kd[3]):
  print(f'[{i}]={k}')

In [ ]:
nomer

In [ ]:
my_print(konk_data()[:60])

In [ ]:
connect = conn()

In [ ]:
disc()

In [ ]:
print(ki)